In [3]:
# Modified from an example at https://dev.to/shamdasani/build-a-flexible-neural-network-with-backpropagation-in-python

import numpy as np

DEBUG = True
NUM_TRAINING_ITERATION = 10

def log(msg = ''):
  if DEBUG is True:
    print(msg)

# training dataset
# x = (hours sleeping, hours studying), y = score on test
rawX = np.array(([2, 9], [1, 5], [3, 6]), dtype=float)
rawY = np.array(([92], [86], [89]), dtype=float)

# scaled values
x = rawX / np.amax(rawX, axis=0) # maximum of X array
y = rawY / 100 # max test score is 100

log('rawX: %s, amax(0): %s, scaledX: %s' % (rawX.tolist(), np.amax(rawX, axis=0).tolist(), x.tolist()))
log('rawY: %s, scaledY: %s' % (rawY.tolist(), y.tolist()))

class Neural_Network(object):
  def __init__(self):
    #parameters
    self.inputSize = 2
    self.outputSize = 1
    self.hiddenSize = 3

    # weights
    self.W1 = np.random.randn(self.inputSize, self.hiddenSize) # (3x2) weight matrix from input to hidden layer
    log('Neural_Network.W1: %s' % (self.W1.tolist()))
    self.W2 = np.random.randn(self.hiddenSize, self.outputSize) # (3x1) weight matrix from hidden to output layer
    print('Neural_Network.W2: %s' % (self.W2.tolist()))

  def forward(self, X):
    # forward propagation through our network
    self.z = np.dot(X, self.W1) # dot product of X (input) and first set of 3x2 weights
    self.z2 = self.sigmoid(self.z) # activation function
    self.z3 = np.dot(self.z2, self.W2) # dot product of hidden layer (z2) and second set of 3x1 weights
    log('Neural_Network.forward(): z: %s, z2: %s, z3: %s' % (self.z.tolist(), self.z2.tolist(), self.z3.tolist()))
    o = self.sigmoid(self.z3) # final activation function
    return o

  def sigmoid(self, s):
    # activation function
    return 1 / (1 + np.exp(-s))

  # slope function
  def sigmoidPrime(self, s):
    # derivative of sigmoid
    return s * (1 - s)

  def backward(self, X, y, o):
    # backward propgate through the network
    log('Neural_Network.backward(): y: %s, o: %s' % (y.tolist(), o.tolist()))

    self.o_error = y - o # error in output
    self.o_delta = self.o_error * self.sigmoidPrime(o) # applying derivative of sigmoid to error
    log('Neural_Network.backward(): o_error: %s, o_delta: %s' % (self.o_error.tolist(), self.o_delta.tolist()))

    self.z2_error = self.o_delta.dot(self.W2.T) # z2 error: how much our hidden layer weights contributed to output error
    self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2) # applying derivative of sigmoid to z2 error
    log('Neural_Network.backward(): z2_error: %s, z2_delta: %s' % (self.z2_error.tolist(), self.z2_delta.tolist()))

    self.W1 += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
    self.W2 += self.z2.T.dot(self.o_delta) # adjusting second set (hidden --> output) weights
    log('Neural_Network.backward(): adjusted weights, w1: %s, w2: %s, ' % (self.W1.tolist(), self.W2.tolist()))

  def train (self, x, y):
    o = self.forward(x)
    self.backward(x, y, o)

def main():
  NN = Neural_Network()
  log()

  for i in range(NUM_TRAINING_ITERATION):
    log('training: (%s)th iteration' % (i + 1))

    prediction = NN.forward(x)
    log('input: %s' % (x.tolist()))
    log('output (ground truth): %s' % (y.tolist()))
    log('prediction: %s' % (prediction.tolist()))
    log('loss: normalizedLoss: %s, rawLoss: %s, rawLossSquared: %s' % (
      np.mean(np.square(y - prediction).tolist()),
      (y - prediction).tolist(),
      np.square(y - prediction).tolist()
    )) # mean sum squared loss
    NN.train(x, y)
    log()

  test1 = np.array(([2, 9]), dtype=float)
  print('test: %s' % (test1.tolist()))
  print('test prediction: %s' % (NN.forward(test1).tolist()))

if __name__ == "__main__":
    main()


 

training: (3)th iteration
Neural_Network.forward(): z: [[-2.649290313632158, -1.3285203481203394, 0.8013928137269575], [-1.433218508740366, -0.6807011033555656, 0.3975117672368434], [-2.345335191583937, -1.7461665827495712, 1.2498588149899665]], z2: [[0.06603276410244856, 0.20940422231094744, 0.6902723386282871], [0.1925976985818167, 0.3361048415586204, 0.5980896889251852], [0.08743727117640743, 0.14853135660441089, 0.7772754204167734]], z3: [[-0.15467472384257125], [-0.26586989348175505], [-0.1999605760618655]]
input: [[0.6666666666666666, 1.0], [0.3333333333333333, 0.5555555555555556], [1.0, 0.6666666666666666]]
output (ground truth): [[0.92], [0.86], [0.89]]
prediction: [[0.4614082283755737], [0.4339213097615759], [0.4501757607847419]]
loss: normalizedLoss: 0.19509827489273382, rawLoss: [[0.4585917716244263], [0.4260786902384241], [0.4398242392152581]], rawLossSquared: [[0.21030641300163], [0.18154305027529094], [0.19344536140128057]]
Neural_Network.forward(): z: [[-2.64929031363